In [1]:
import pandas as pd
from Bio import SeqIO; from Bio.Seq import Seq; from Bio.SeqRecord import SeqRecord; from Bio.SeqFeature import SeqFeature, FeatureLocation
import os
import re

In [2]:
# latest paftol export containing ENA sample accessions
# https://www.ebi.ac.uk/ena/submit/report/swagger-ui.html#/samples/findAllUsingGET_13
pft_file = '../PAFTOL_DB/2022-03-14_paftol_export_stats.csv'
cptree_file = 'cpTree_v7/cpTree_v7_tree_clean.csv'
org_dir = 'C:/Data/PAFTOL/Organelles/'

# DataSource = 'GAP' 
# ENA_project_number = 'PRJEB49212'
DataSource = 'PAFTOL' 
ENA_project_number = 'PRJEB35285' #https://www.ncbi.nlm.nih.gov/bioproject/588607

In [3]:
## All Validated Samples
R = pd.read_csv(cptree_file)
R = R[R.DataSource==DataSource]
subset = R.Sample_Name.to_list()
print(R.shape[0],subset[:2])

3451 ['PAFTOL_004799', 'PAFTOL_000992']


## Load data and subset

In [4]:
pdir = DataSource + '/'
sdir = 'ENA_submissions/' + pdir
if not os.path.exists(sdir):
    os.makedirs(sdir)

In [5]:
# Load GetOrg result table
Org_df = pd.read_csv(org_dir + DataSource + '/' + DataSource + '_Organelle_Recovery.csv').astype({'idSequencing':'int'})
if DataSource == 'PAFTOL':
    Org_df['SeqID'] = Org_df.Sample_Name.str.replace('PAFTOL_','Pis_')
elif DataSource in ['GAP']:
    Org_df['SeqID'] = Org_df.Sample_Name
elif DataSource in ['SRA']:
    sra2_pt = pd.read_csv('cpSRA_GetOrg/cpSRA_GetOrg_Organelle_Recovery.csv')
    Org_df = pd.concat([Org_df,sra2_pt],ignore_index=True)
    Org_df = Org_df.groupby('Sample_Name').head(1)
    Org_df['SeqID'] = Org_df.Sample_Name
print(Org_df.shape[0])
Org_df[:2]

9715


,idPaftol,idSequencing,Sample_Name,Project,order,family,genus,species,Taxonomical_Notes,SumContigLength,...,Coverage_Kmer,Coverage_base,Run_Time,maxK,NRepeat_Pattern,NPath,Redo_FastPlast,error_pt,error_nr,SeqID
0,2,961,PAFTOL_000961,Pilot,Ranunculales,Circaeasteraceae,Circaeaster,agrestis,NaN,138795.0,...,59.4,90.7,329.43,105.0,NaN,NaN,False,NaN,NaN,Pis_000961
1,4,2571,PAFTOL_002571,Connaraceae,Oxalidales,Connaraceae,Rourea,minor,NaN,213954.0,...,18.5,60.2,747.57,105.0,NaN,NaN,False,NaN,NaN,Pis_002571


In [6]:
if subset is not None:
    Org_df = Org_df[Org_df.Sample_Name.isin(subset)]
    print(Org_df.shape[0],Org_df.Sample_Name.nunique())

3451 3451


In [7]:
# Load paftol export and merge ena sample IDs
db = pd.read_csv(pft_file)
db = db[db.idPaftol.notnull()].astype({'idPaftol':'int','idSequencing':'int'})
db = db[['Sample_Name','idPaftol', 'DataSource', 'Project', 'Order', 'Family', 'Genus', 'Species', 'Taxonomical_Notes', 
         'ENASampleNum', 'idSequencing', 'ExternalSequenceID', 'ENAExpNumber',
       'ENARunNumber', 'NumReads', 'NumRecoveredGenes', 'SumContigLength','NCBI_TaxID', 'NCBI_sciname']]
#        'title', 'taxId','id', 'alias', 'firstCreated', 'firstPublic', 'releaseStatus', 'secondaryId', 'scientificName', 'commonName']]
db[:2]

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (29,35,45,57,74,83) have mixed types. Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Sample_Name,idPaftol,DataSource,Project,Order,Family,Genus,Species,Taxonomical_Notes,ENASampleNum,idSequencing,ExternalSequenceID,ENAExpNumber,ENARunNumber,NumReads,NumRecoveredGenes,SumContigLength,NCBI_TaxID,NCBI_sciname
0,PAFTOL_000564,1,PAFTOL,Asteraceae,Asterales,Asteraceae,Gymnolaena,sp.,Initially recorded as Gymnolaena litoralis but...,NaN,564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,PAFTOL_000961,2,PAFTOL,Pilot,Ranunculales,Circaeasteraceae,Circaeaster,agrestis,NaN,ERS5501508,961,NaN,ERX4839538,ERR5033191,625228.0,332.0,138795.0,39288.0,Circaeaster agrestis


In [8]:
Org_df = pd.merge(Org_df,
         db[['Sample_Name','DataSource','ENASampleNum','ENAExpNumber','ENARunNumber','NCBI_TaxID','NCBI_sciname']],
        how='left',on='Sample_Name').groupby('Sample_Name').head(1)
print(Org_df.isna().sum().to_dict())
print(Org_df.shape[0],Org_df.Sample_Name.nunique())

{'idPaftol': 0, 'idSequencing': 0, 'Sample_Name': 0, 'Project': 0, 'order': 0, 'family': 0, 'genus': 0, 'species': 0, 'Taxonomical_Notes': 2231, 'SumContigLength': 363, 'sci_name': 0, 'log_pt': 0, 'log_nr': 0, 'pt_recovered': 0, 'Nseq_pt': 0, 'Sum_len_pt': 0, 'Nanybase_pt': 0, 'nr_recovered': 0, 'Nseq_nr': 27, 'Sum_len_nr': 27, 'Nanybase_nr': 27, 'Completed_Reads': 0, 'Completed_Extending': 0, 'Completed_Assembly': 0, 'Completed_Slimming': 0, 'Completed_Output': 0, 'Warning_Look': 0, 'Warning_Multi_Structure': 0, 'Warning_Self_Loop': 0, 'Info_Large_Repeats': 0, 'Info_Disentangling_Failed': 0, 'Assembly': 6, 'Coverage_Kmer': 635, 'Coverage_base': 8, 'Run_Time': 5, 'maxK': 6, 'NRepeat_Pattern': 3380, 'NPath': 3422, 'Redo_FastPlast': 0, 'error_pt': 3450, 'error_nr': 3432, 'SeqID': 0, 'DataSource': 0, 'ENASampleNum': 47, 'ENAExpNumber': 0, 'ENARunNumber': 0, 'NCBI_TaxID': 172, 'NCBI_sciname': 172}
3451 3451


In [9]:
geseq_dir = sdir + 'fasta/'
if not os.path.exists(geseq_dir):
    os.makedirs(geseq_dir)

# Whole plastomes
### Annotation

In [ ]:
whole_pt = Org_df[(Org_df.Assembly.isin(['circular genome','circular genome with gaps']) & (Org_df.Sum_len_pt>80000))]
print(whole_pt.shape[0],'samples',whole_pt.genus.nunique(),'genera')
whole_pt[:2]

In [ ]:
# Output fasta files 
for idx, row in whole_pt.iterrows():
    seqs = list(SeqIO.parse(org_dir + DataSource + '/' + 'fasta_pt/' + row.Sample_Name + '_pt.fasta',format='fasta'))
    # Verif only 1 sequence
    if len(seqs) > 1:
        print('error, more than 1 sequence')
        pass
    seq = seqs[0]
    print(row.Sample_Name, row.SeqID, row.family, row.sci_name,len(seq),seq.seq.count('N'))
    seq.id = row.SeqID
    seq.description = ''
    SeqIO.write(seq, geseq_dir + row.SeqID + '-circular.fasta',format='fasta')

Submit files to GeSeq https://chlorobox.mpimp-golm.mpg.de/geseq.html and extract zip file in wdir/GeSeq_output

In [ ]:
whole_pt.to_csv(sdir + DataSource + '_wholePT.csv',index=False)

# Contigs

In [10]:
references = pd.read_csv('References/NCBI_plastid_ref/NCBI_plastid_ref_list_SELECTED.csv')
print(references.shape)
references.head(2)

(124, 24)


,SeqID,Ini_Species,TaxID,mol_type,length,Ini_sci_name,sci_name_query,Similar_match,Duplicates,kew_id,...,sci_name,authors,rank,taxonomic_status,Ini_kew_id,Ini_taxonomic_status,Duplicate_type,Nsp,class,order
0,NC_045294.1,Acorus tatarinowii,123564,genomic DNA,153296,Acorus tatarinowii,Acorus tatarinowii,NaN,False,77188885-1,...,Acorus calamus var. angustatus,Besser,VARIETY,Accepted,84028-1,Synonym,NaN,2,Magnoliopsida,Acorales
1,NC_053912.1,Aponogeton lakhonensis,217757,genomic DNA,154860,Aponogeton lakhonensis,Aponogeton lakhonensis,NaN,False,82954-1,...,Aponogeton lakhonensis,A.Camus,SPECIES,Accepted,NaN,NaN,NaN,62,Magnoliopsida,Alismatales


In [17]:
incomplete_pt = Org_df[(Org_df.Assembly.isin(['circular genome','circular genome with gaps'])==False) &
                   (Org_df.pt_recovered==True)]
print(incomplete_pt.shape[0],'samples',incomplete_pt.genus.nunique(),'genera')
incomplete_pt = incomplete_pt[incomplete_pt.Sum_len_pt<200000].set_index('Sample_Name')
print(incomplete_pt.shape[0],'samples',incomplete_pt.genus.nunique(),'genera')
### Random subset for test 
incomplete_pt = incomplete_pt.sample(n=100)
print(incomplete_pt.shape[0],'samples',incomplete_pt.genus.nunique(),'genera')
incomplete_pt[:2]

3114 samples 2623 genera
3114 samples 2623 genera
100 samples 100 genera


,idPaftol,idSequencing,Project,order,family,genus,species,Taxonomical_Notes,SumContigLength,sci_name,...,Redo_FastPlast,error_pt,error_nr,SeqID,DataSource,ENASampleNum,ENAExpNumber,ENARunNumber,NCBI_TaxID,NCBI_sciname
Sample_Name,,,,,,,,,,,,,,,,,,,,,
PAFTOL_007905,4838,7905,Celastraceae,Celastrales,Celastraceae,Euonymus,australianus,Euonymus-Celastrus-Haydenoxylon clade,201111.0,Euonymus australianus,...,False,NaN,NaN,Pis_007905,PAFTOL,ERS8702347,ERX7189840,ERR7619260,1089391.0,Euonymus australianus
PAFTOL_014479,8655,14479,Asteraceae,Asterales,Asteraceae,Olgaea,leucophylla,NaN,182634.0,Olgaea leucophylla,...,False,NaN,NaN,Pis_014479,PAFTOL,ERS8704497,ERX7191722,ERR7621142,591993.0,Olgaea leucophylla


In [21]:
references[references.order=='Sapindales']

,SeqID,Ini_Species,TaxID,mol_type,length,Ini_sci_name,sci_name_query,Similar_match,Duplicates,kew_id,...,sci_name,authors,rank,taxonomic_status,Ini_kew_id,Ini_taxonomic_status,Duplicate_type,Nsp,class,order
109,NC_048982.1,Canarium album,300208,genomic DNA,163347,Canarium album,Canarium album,NaN,True,127264-1,...,Canarium album,(Lour.) DC.,SPECIES,Accepted,NaN,NaN,Same_Taxon,122,Magnoliopsida,Sapindales
110,NC_024929.1,Citrus aurantiifolia,159033,genomic DNA,159893,Citrus aurantiifolia,Citrus × aurantiifolia,True,False,59599-2,...,Citrus × aurantiifolia,(Christm.) Swingle,SPECIES,Accepted,NaN,NaN,NaN,94,Magnoliopsida,Sapindales


In [22]:
order_dc = {'Gunnerales':'Dilleniales','Picramniales':'Sapindales'}

In [23]:
commands = []
for Sample_Name, row in incomplete_pt.iterrows():
    print(Sample_Name,row.order,row.family)
    ### Identify best reference
    # check if reference in same family
    best_ref = references[references.family==row.family]
    if best_ref.shape[0]<1:
        # otherwise order
        best_ref = references[references.order==row.order]
        if best_ref.shape[0]<1:
            #Otherwise use closest order
            best_ref = references[references.order==order_dc[row.order]]
    print('\t Ref',best_ref.shape, end = ': ')
    best_ref = best_ref.iloc[0]
    print(best_ref.SeqID,best_ref.order,best_ref.family)
    incomplete_pt.loc[Sample_Name,'Ref-SeqID'] = best_ref.SeqID
    incomplete_pt.loc[Sample_Name,'Ref-order'] = best_ref.order
    incomplete_pt.loc[Sample_Name,'Ref-family'] = best_ref.family
    
    ### launch blastn
    # program_path = 'C:/Program Files/NCBI/blast-BLAST_VERSION+/bin/blastn.exe'
    program_path = 'blastn'
    ref_path = 'References/NCBI_plastid_ref/' + best_ref['SeqID'] + '.fasta'
    contigs_path = '../Organelles/' + DataSource + '/fasta_pt/' + Sample_Name + '_pt.fasta'
    out_path = 'ENA_submissions/' + DataSource + '/out_blast/'+ Sample_Name + '.blast'
    command = program_path + ' -query ' + contigs_path + ' -subject ' + ref_path + \
                ' -outfmt "6 qseqid sseqid pident length slen qlen mismatch gapopen qstart qend sstart send evalue bitscore"' + \
                ' -out ' + out_path
    commands.append(command)
    os.system(command)
    
    ### read blast output
    blast_output = pd.read_table(out_path, sep='\t',header=None)
    blast_output.columns = ['qseqid','sseqid','pident','length','slen','qlen','mismatch','gapopen','qstart','qend','sstart','send','evalue','bitscore']
#     print('\t',blast_output.shape[0],'matched for', blast_output.qseqid.nunique(),'qseqid',end='; ')
    incomplete_pt.loc[Sample_Name,'Blast-N_qseqid_match'] = blast_output.qseqid.nunique()
    # Keep best match (by bitscore) per queried contig
    blast_output = blast_output.sort_values('bitscore',ascending=False).groupby('qseqid').head(1)
    blast_output['oriented_sstart'] = blast_output[['sstart','send']].min(axis=1); blast_output['oriented_ssend'] = blast_output[['sstart','send']].max(axis=1)
    blast_output['Need_RevComp'] = blast_output.sstart > blast_output.oriented_sstart
    blast_output = blast_output.sort_values('oriented_sstart').set_index('qseqid')
#     print('Contigs to reverse complement:',blast_output.Need_RevComp.value_counts().to_dict())
    
    
    ### Load contigs and write concatenated fasta
    gapN = 100
    records = SeqIO.to_dict(SeqIO.parse(contigs_path, "fasta"))
    print('\t',blast_output.shape[0],'/',len(records),'contigs mapped to reference')
    concat_seq = ''
    for qseqid, blast_row in blast_output.iterrows():
        if blast_row.Need_RevComp==True:
            concat_seq += records[qseqid].seq.reverse_complement()
        else:
            concat_seq += records[qseqid].seq 
        concat_seq += Seq(gapN * 'N')
    concat_seq = concat_seq[:-gapN] # Remove trailing Ns
    print('\t sum len concat:',blast_output.qlen.sum(),'/',int(row.Sum_len_pt),round(blast_output.qlen.sum()/int(row.Sum_len_pt)*100,1),'%')
    incomplete_pt.loc[Sample_Name,'Concat-len'] = blast_output.qlen.sum()
    incomplete_pt.loc[Sample_Name,'Concat-%_mapped'] = round(blast_output.qlen.sum()/int(row.Sum_len_pt)*100,1)
    incomplete_pt.loc[Sample_Name,'Concat-len_with_Ns'] = len(concat_seq)
    Concat_record = SeqRecord(
        concat_seq,
        id=row.SeqID + '_concat_pt',
        name=Sample_Name + '_concat_pt',
        description=''
    )
    SeqIO.write(Concat_record,'ENA_submissions/' + DataSource + '/fasta/' + row.SeqID + '-concat.fasta','fasta')

PAFTOL_007905 Celastrales Celastraceae
	 Ref (1, 24): NC_045280.1 Celastrales Celastraceae
	 30 / 31 contigs mapped to reference
	 sum len concat: 24986 / 25415 98.3 %
PAFTOL_014479 Asterales Asteraceae
	 Ref (1, 24): NC_044728.1 Asterales Asteraceae
	 30 / 30 contigs mapped to reference
	 sum len concat: 36976 / 36976 100.0 %
PAFTOL_006622 Fabales Fabaceae
	 Ref (1, 24): NC_025745.1 Fabales Fabaceae
	 9 / 10 contigs mapped to reference
	 sum len concat: 157832 / 158427 99.6 %
PAFTOL_010719 Sapindales Meliaceae
	 Ref (2, 24): NC_048982.1 Sapindales Burseraceae
	 26 / 26 contigs mapped to reference
	 sum len concat: 54070 / 54070 100.0 %
PAFTOL_012011 Ericales Sapotaceae
	 Ref (2, 24): NC_041127.1 Ericales Styracaceae
	 8 / 8 contigs mapped to reference
	 sum len concat: 127068 / 127068 100.0 %
PAFTOL_004680 Asparagales Asphodelaceae
	 Ref (2, 24): NC_053784.1 Asparagales Asparagaceae
	 47 / 47 contigs mapped to reference
	 sum len concat: 58789 / 58789 100.0 %
PAFTOL_018841 Rosales Ros

In [24]:
incomplete_pt.to_csv(sdir + DataSource + '_concatPT.csv')

### ENA submission
https://ena-docs.readthedocs.io/en/latest/submit/fileprep/assembly.html?highlight=chloroplast#

cd C:\Data\PAFTOL\cpDNA\ENA_submissions

In [47]:
paper_title = 'Building the largest, curated plastid genome dataset and phylogeny by leveraging nuclear-oriented sequence data'
paper_authors = 'KEVIN LEEMPOEL1, PAUL BAILEY1, ELAINE FRANCOSO1, BERTA GALLEGO1, De-Zhu Li2,3,4, Hong-Tao Li2,3, OLIVIER MAURIN1, SHYAMALI ROY1,' +  \
            'Jun-Bo Yang2,3, Ting-Shuang Yi2,3, ALEXANDRE R. ZUNTINI1, WOLF L. EISERHARDT1,5,' +  \
            'PAUL J. KERSEY1, ILIA J. LEITCH1, FELIX FOREST1 AND WILLIAM J. BAKER1'
paper_journal = 'Unpublished'
paper_id = 'NA'

In [48]:
submit_pt = pd.concat([pd.read_csv(sdir + DataSource + '_wholePT.csv'), pd.read_csv(sdir + DataSource + '_concatPT.csv')],ignore_index=True)
# submit_pt = pd.read_csv(sdir + DataSource + '_concatPT.csv')
print(submit_pt.shape[0])
submit_pt = submit_pt[(submit_pt.ENASampleNum.notnull()) & (submit_pt.ENARunNumber.notnull())]
print(submit_pt.shape[0])
submit_pt = submit_pt[submit_pt.NCBI_sciname.notnull()]
print(submit_pt.shape[0],submit_pt.Assembly.value_counts().to_dict())
submit_pt.head(2)

437
428
410 {'circular genome': 315, 'scaffold': 93, 'circular genome with gaps': 2}


<ipython-input-48-9115cba50240>:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  submit_pt = pd.concat([pd.read_csv(sdir + DataSource + '_wholePT.csv'), pd.read_csv(sdir + DataSource + '_concatPT.csv')],ignore_index=True)


,Assembly,Blast-N_qseqid_match,Completed_Assembly,Completed_Extending,Completed_Output,Completed_Reads,Completed_Slimming,Concat-%_mapped,Concat-len,Concat-len_with_Ns,...,idPaftol,idSequencing,log_nr,log_pt,maxK,nr_recovered,order,pt_recovered,sci_name,species
0,circular genome,NaN,True,True,True,True,True,NaN,NaN,NaN,...,2,961,True,True,105.0,True,Ranunculales,True,Circaeaster agrestis,agrestis
1,circular genome,NaN,True,True,True,True,True,NaN,NaN,NaN,...,11,12989,True,True,105.0,True,Malpighiales,True,Gynocardia odorata,odorata


In [57]:
submit_pt = submit_pt[submit_pt.ENASampleNum=='ERS10966407']

In [58]:
all_seq = pd.read_csv('cpTree_v7/AllSamples_Allgenes_All_SeqTable.csv')
print(all_seq.shape)
genes_82 = pd.read_table('cp_genes_82.txt',header=None).rename(columns={0:'gene'})
all_seq = all_seq[all_seq.gene.isin(genes_82.gene)]
print(all_seq.shape)
all_seq.head(2)

(1678470, 4)
(1172782, 4)


,seqid,len,Sample_Name,gene
0,GAP_026547-rrn23,2812,GAP_026547,rrn23
1,GAP_026547-rrn16,1491,GAP_026547,rrn16


In [59]:
def list_genes(record, ls_types = ['gene','CDS','rRNA']):
    # list gene names
    feat_ls = []; idx = 0
    for feature in record.features:
        if feature.type in ls_types:
            feat_ls.append(feature.qualifiers['gene'][0])
        idx += 1
    return list(set(feat_ls))

In [92]:
# Rename and modify genbank files, convert to embl flat file
if not os.path.exists(sdir + '/Genbank_files/'):
    os.makedirs(sdir + '/Genbank_files/')
gb_files = [file for file in os.listdir(sdir + 'geseq/') if file.endswith('.gb')]
print(len(gb_files))
for idx, row in submit_pt.iterrows():
    idx_file = [i for i, x in enumerate([row.SeqID in file for file in gb_files]) if x]
    print(row.Sample_Name,gb_files[idx_file[0]],end=':')
    records = list(SeqIO.parse(sdir + 'geseq/' + gb_files[idx_file[0]], "genbank"))
    record = records[0]
    record_genes = list_genes(record); record_genes = [gene for gene in record_genes if gene in genes_82.gene.to_list()]
    blast_genes = all_seq[all_seq.Sample_Name==row.Sample_Name]
    print(len(records), len(records[0]), len(record_genes),'/',blast_genes.gene.nunique(),
         int(len(record_genes)/blast_genes.gene.nunique()*100),'%',blast_genes[blast_genes.gene.isin(record_genes)==False].gene.unique())
    try:
        if row.Assembly in ['circular genome','circular genome with gaps']:
            record.description = row.ENARunNumber + ' chloroplast, complete genome'
        elif row.Assembly in ['scaffold']:
            record.description = row.ENARunNumber + ' chloroplast, partial genome (concatenated contigs)'
        submit_pt.loc[idx,'ENA_SeqID'] = row.ENARunNumber + '_pt'
        record.id = submit_pt.loc[idx,'ENA_SeqID']
        record.name = submit_pt.loc[idx,'ENA_SeqID']
        record.version = submit_pt.loc[idx,'ENA_SeqID'] + '.1'
        record.annotations['source'] = row.ENASampleNum + ', ' + row.sci_name + ', isolate'
        record.annotations['accessions'] = row.ENASampleNum
        record.annotations['organism'] = row.NCBI_sciname
        record.annotations["data_file_division"]="PLN";
        record.annotations.pop('taxonomy')
        record.annotations['references'][0].title = paper_title    
        record.annotations['references'][0].authors = paper_authors    
        record.annotations['references'][0].journal = paper_journal      
        record.annotations['references'][0].pubmed_id = paper_id    
        record.features[0].qualifiers['organism'] = row.NCBI_sciname
        record.features[0].qualifiers['db_xref'] = ['taxon:' + str(int(row['NCBI_TaxID']))]
        SeqIO.write(record,sdir + 'Genbank_files/' + row.SeqID + '_pt.gb',format='genbank')
    except:
        print('issue with ',gb_files[idx_file[0]])

439
PAFTOL_029857 GeSeqJob-20220305-120228_Pis_029857_concat_pt_GenBank.gb:1 31862 20 / 15 133 % ['atpA']


In [93]:
def del_features(record, feat_del_ls = ['translation','info','annotator']):
    for feature in record.features:
        for feat_to_del in feat_del_ls:
            if feat_to_del in feature.qualifiers:
                del feature.qualifiers[feat_to_del]
    return record

def del_fragment(record, ls_types = ['gene','CDS','rRNA']):
    # list gene names
    ls_feat = []
    for feature in record.features:
        if feature.type in ls_types:
            ls_feat.append(feature.qualifiers['gene'][0])
    ls_feat = [ifeat for ifeat in ls_feat if 'fragment' not in ifeat]

    # remove fragment feature if full feature is present
    rm_count = 0
    for feature in record.features:
        if feature.type in ls_types:
            if 'fragment' in feature.qualifiers['gene'][0]:
                if feature.qualifiers['gene'][0].split('-')[0] in ls_feat:
                    rm_count += 1
                    record.features.remove(feature)
    print('removed',rm_count,'partial features')
    return record

def del_longfeat(record, ls_types = ['gene','CDS','rRNA'], len_thrs = 5000):
    for feature in record.features:
        if feature.type in ls_types:
            if len(feature)>len_thrs:
                print('removing ',feature.qualifiers['gene'],len(feature))
                record.features.remove(feature)
    return record

def add_gap_features(record, gapN = 100):
    # find gaps in record
    gaps_idx = [m.start() for m in re.finditer(gapN * 'N', str(record.seq))]
    print(len(gaps_idx),'gaps')
    if len(gaps_idx)>0:
        ## for each gap, add a sequence feature
        for gap_pos in gaps_idx:
            # find index of gap in features
            features_start = [feat.location.start for feat in record.features]
            gap_feature_idx = [i for i in range(len(features_start)) if features_start[i] > gap_pos]
            
            gap_feat = SeqFeature(FeatureLocation(gap_pos,gap_pos+gapN), strand=1, type='assembly_gap')
            gap_feat.qualifiers['gap_type']='within scaffold'; 
            gap_feat.qualifiers['estimated_length']='unknown'
            gap_feat.qualifiers['linkage_evidence']='unspecified'
            if len(gap_feature_idx)>0:
                gap_feature_idx = gap_feature_idx[0]
                record.features.insert(gap_feature_idx, gap_feat)
            elif len(gap_feature_idx)==0: #if position is further than any other feature, append at the end of feature list
                print("END")
                record.features.append(gap_feat)
    return record

def gzip_file(in_file):
    import gzip
    import shutil
    f_gz = in_file + '.gz'
    with open(in_file, 'rb') as f_in:
        with gzip.open(f_gz, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [95]:
# ENA files
if not os.path.exists(sdir + '/EMBL_files/'):
    os.makedirs(sdir + '/EMBL_files/')
commands = []
for idx, row in submit_pt.iterrows():
    ### Convert .gb to EMBL flat files
    file_gb = sdir + 'Genbank_files/' + row.SeqID + '_pt.gb'
    print(row.SeqID,row.ENA_SeqID,end=' > ')
#     try:
    record = list(SeqIO.parse(file_gb, "genbank"))[0]
    print(len(record))
    record_clean = del_features(record = record)
    record_clean = del_fragment(record = record_clean); record_clean = del_fragment(record = record_clean);
    record_clean = del_longfeat(record = record_clean,len_thrs=8000);
    record_clean = add_gap_features(record = record_clean)
    file_embl = sdir + '/EMBL_files/' + row.ENA_SeqID + '.embl'
    SeqIO.write(record_clean, file_gb, "genbank")
    SeqIO.write(record_clean, file_embl, "embl")
    gzip_file(file_embl)

    ### Chromosome list file
    # https://ena-docs.readthedocs.io/en/latest/submit/fileprep/assembly.html?highlight=chloroplast#chromosome-list-file
    if row.Assembly in ['circular genome','circular genome with gaps']:
        list_file=row.ENA_SeqID + '	PT	Circular-Chromosome	Chloroplast'       
    elif row.Assembly in ['scaffold']:
        list_file=row.ENA_SeqID + '	PT	Linear-Chromosome	Chloroplast'
    with open(file_embl.replace('.embl','_CHR.txt'), 'wb') as f:
        f.write(list_file.encode("ascii"))
    gzip_file(file_embl.replace('.embl','_CHR.txt'))

    ### Manifest file 
    # https://ena-docs.readthedocs.io/en/latest/submit/assembly/genome.html 
    manifest_file = file_embl.replace('.embl','_manifest.txt')
    if row.Assembly in ['circular genome','circular genome with gaps']:
        tmp_description=' chloroplast, complete genome' 
    elif row.Assembly in ['scaffold']:
        tmp_description=' chloroplast, partial genome (concatenated contigs)' 
    manifest={
        'STUDY':ENA_project_number,
        'SAMPLE':row.ENASampleNum,
        'ASSEMBLYNAME':row.ENA_SeqID,
        'ASSEMBLY_TYPE':'clone or isolate',
        'COVERAGE':row.Coverage_base,
        'PROGRAM':'GetOrganelle v1.7.5 +  GeSeq 2.03',
        'PLATFORM':'ILLUMINA',
        'MOLECULETYPE':'genomic DNA',
        'FLATFILE': row.ENA_SeqID + '.embl.gz',
        'CHROMOSOME_LIST': row.ENA_SeqID + '_CHR.txt.gz',
        'DESCRIPTION': row.NCBI_sciname + tmp_description
    }
    manifest = pd.DataFrame.from_dict(manifest,orient='index')
    manifest.to_csv(manifest_file,header=None,sep='\t')

    # Submission command
    commands.append('java -jar C:/Data/PAFTOL/ENA_submissions/webin-cli-4.3.0.jar ' + 
          '-username Webin-52995 -passwordFile C:/Data/PAFTOL/ENA_submissions/ena_pwd.txt -context genome -manifest ' +
          row.ENA_SeqID + '_manifest.txt' + ' -validate') 
#     except:
#         print('issue',row.SeqID)
pd.DataFrame(commands).to_csv(sdir + '/ENA_Submission_Commands_' + DataSource + '.txt',index=False,header=None)

Pis_029857 ERR9230283_pt > 31862
removed 1 partial features
removed 1 partial features
28 gaps


In [77]:
file_gb

'ENA_submissions/PAFTOL/Genbank_files/Pis_029857_pt.gb'

In [72]:
record_clean.features

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(31862), strand=1), type='source'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(31310), ExactPosition(31862), strand=1), FeatureLocation(ExactPosition(0), ExactPosition(6), strand=1)], 'join'), type='gene', location_operator='join'),
 SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(31310), ExactPosition(31862), strand=1), FeatureLocation(ExactPosition(0), ExactPosition(6), strand=1)], 'join'), type='CDS', location_operator='join'),
 SeqFeature(FeatureLocation(ExactPosition(615), ExactPosition(1334), strand=-1), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(615), ExactPosition(1334), strand=-1), type='intron'),
 SeqFeature(FeatureLocation(ExactPosition(1727), ExactPosition(1973), strand=-1), type='gene'),
 SeqFeature(FeatureLocation(ExactPosition(1727), ExactPosition(1973), strand=-1), type='CDS'),
 SeqFeature(FeatureLocation(ExactPosition(2017), ExactPosition(2359), strand=-1), type='gene'),

In [54]:
pd.DataFrame(commands).to_csv(sdir + '/ENA_Submission_Commands_' + DataSource + '.txt',index=False,header=None)